In [ ]:
import spidev
import time
import RPi.GPIO as GPIO
import math
import json
import mysql.connector
import paho.mqtt.client as mqtt

# GPIO Pin for Manual CS
CS_PIN = 26
relay_pin = 17
scaling_factor = 240 / 3.3  # ZMPT sensor scaling
pf = 0.87  # Power factor
tariff = 4  # Rs per kWh

mqtt_broker = "localhost"
mqtt_port = 1883
mqtt_topic = "DC"
client = mqtt.Client(mqtt.CallbackAPIVersion.VERSION2)
client.connect(mqtt_broker, mqtt_port, 60)

# Setup GPIO
GPIO.setmode(GPIO.BCM)
GPIO.setup(CS_PIN, GPIO.OUT)
GPIO.output(CS_PIN, GPIO.HIGH)  # Set CS High initially
GPIO.setup(relay_pin, GPIO.OUT)


# Create SPI object for SPI0
spi = spidev.SpiDev()
spi.open(0, 0)  # SPI0 (Bus 0)
spi.max_speed_hz = 1350000  # 1.35 MHz SPI speed

# MySQL Database Connection
db = mysql.connector.connect(
    host="localhost",
    user="pi",
    password="your_password",  # Add your MySQL password if set
    database="sensor_data"
)
cursor = db.cursor()

# Function to read data from MCP3008
def read_adc(channel):
    GPIO.output(CS_PIN, GPIO.LOW)  # CS LOW
    adc = spi.xfer2([1, (8 + channel) << 4, 0])
    GPIO.output(CS_PIN, GPIO.HIGH)  # CS HIGH
    value = ((adc[1] & 3) << 8) + adc[2]  # 10-bit ADC value
    return value

# Function to calculate RMS Voltage
def calculate_rms(channel, num_samples=100):
    values = []
    for _ in range(num_samples):
        adc_value = read_adc(channel)
        voltage = adc_value * 3.3 * scaling_factor / 1023  # Convert ADC value to voltage
        values.append(voltage)
        time.sleep(0.01)
    
    squared_diff = [v ** 2 for v in values]
    return math.sqrt(sum(squared_diff) / num_samples)

# Function to fetch relay state from the database
def get_relay_state():
    cursor.execute("SELECT relay FROM readings ORDER BY id DESC LIMIT 1")
    result = cursor.fetchone()
    return result[0] if result else 0  # Default to OFF if no result

# Start Timer
start_time = time.time()
total_energy = 0  # Total energy consumed in kWh

while True:
    # Get the relay state from the database
    relay_state = get_relay_state()
    print("state: ",relay_state)

    # Control the relay and LED based on the relay state
    if relay_state == 0:  # Relay OFF
        GPIO.output(relay_pin, GPIO.LOW)  # Turn OFF relay (bulb)
        
        voltage = 0  # Set voltage to 0 when relay is off
        print("Voltage0: ",voltage)
    else:  # Relay ON
        GPIO.output(relay_pin, GPIO.HIGH)  # Turn ON relay (bulb)
        
        voltage_rms = calculate_rms(0)  # RMS Voltage from Channel 0
        voltage = voltage_rms  # Set voltage to measured value when relay is on
        print("Voltage1: ",voltage)

    # Current Measurement
    num_samples = 100
    current_values = []
    
    for _ in range(num_samples):
        adc_value = read_adc(1)
        current = adc_value * 3.3 / 1023  # Convert ADC value to voltage
        current = (current - 2.5) / 0.066  # Adjusting offset for ACS712
        current_values.append(current)
        time.sleep(0.01)

    # Remove DC Component
    mean_current = sum(current_values) / num_samples
    squared_diff_current = [(i - mean_current) ** 2 for i in current_values]
    current_rms = math.sqrt(sum(squared_diff_current) / num_samples)

    # Power Calculation
    power = voltage * current_rms * pf  # Active Power (Watts)

    # Energy Consumption
    elapsed_time = (time.time() - start_time) / 3600  # Convert to hours
    energy_kwh = (power * elapsed_time) / 1000  # Convert to kWh
    total_energy += energy_kwh  # Accumulate total energy

    # Bill Calculation
    bill = total_energy * tariff  # Cost in Rs

   
    data = {
        "Voltage": round(voltage, 3),
        "Energy_kWh": round(total_energy, 3),
        "Bill_Rs": round(bill, 2)
    }

    cursor.execute("SELECT id FROM readings ORDER BY id DESC LIMIT 1")
    latest = cursor.fetchone()

    if latest:
        latest_id = latest[0]
        update_query = "UPDATE readings SET voltage=%s, energy=%s, bill=%s, relay=%s WHERE id=%s"
        cursor.execute(update_query, (
            data["Voltage"], data["Energy_kWh"], data["Bill_Rs"], relay_state, latest_id
    ))
        db.commit()


    # Publish to MQTT
    json_data = json.dumps(data)
    client.publish(mqtt_topic, json_data)

    # Adjust as needed
    print(data)
  
